# 1. General notes

### Local CTS server

The public Scaife CTS server from Perseus doesn't provide Quintus. The text exists in the [canonical-greekLit](https://github.com/PerseusDL/canonical-greekLit) Git repo, but it's not configured for Nautilus to serve it. I created my own [canonical-greekLit fork](https://github.com/cwf2/canonical-greekLit) and edited Quintus until Nautilus was happy.

This notebook should come with cached data, so you don't need to reprocess the texts. If you do want to replicate everything from scratch, then run the following code in a terminal window to install and run the CTS server locally.

```bash
    git clone https://github.com/cwf2/canonical-greekLit
    capitains-nautilus canonical-greekLit --port 5000
```

### Odyssey variant reading

The DICES database has a speech by Circe to Odysseus beginning at Od. 10.456; but in the Perseus edition, 456 is missing and the speech begins at 457. I've manually changed the speech start line here to agree with Perseus, avoiding an error when we download the text.

# 2. Steps for processing the speeches

## Import statements

In [ ]:
from dicesapi import DicesAPI
from dicesapi.jupyter import NotebookPBar
from dicesapi.text import CtsAPI
from IPython.display import display
from ipywidgets import interactive, widgets
import pandas as pd
import os
import pickle

## Initialize connections to DICES and CTS server

In [ ]:
# initialize connection to the database
api = DicesAPI(logfile='dices.log')

# initialize connection to digital libraries
cts = CtsAPI(
    dices_api = api,
    servers = {
        # None:  'https://scaife-cts.perseus.org/api/cts', # default
        None: 'http://localhost:5000/cts', # use local server
    }
)

## Workflow

I'm setting this up the steps as a series of function definitions so that it's easier to loop over the individual texts.

### Download the speech metadata

In [ ]:
def dlSpeechData(work):
    '''Download all the speeches for a given work'''

    print(f'Retrieving speeches for {work}')
            
    speeches = api.getSpeeches(work_title=work.title())
    print('retrieved', len(speeches), 'results')

    
    # cludge for textual variant in Odyssey
    if work.title() == 'Odyssey':
        for s in speeches:
            if s.l_fi == '10.456':
                s.l_fi = '10.457'
    
    # another cludge to remove the apologia
    if work.title() == 'Odyssey':
        speeches = [s for s in speeches if s.l_fi.split('.')[0] == s.l_la.split('.')[0]]
                
    return speeches

### Download the text of the speeches

In [ ]:
def dlSpeechText(speeches):
    '''Download the text of the speeches from CTS server, append to speech objects'''
    pbar = NotebookPBar(max=len(speeches), prefix='Downloading text')

    for s in speeches:
        if not hasattr(s, 'passage') or s.passage is None:
            s.passage = cts.getPassage(s)
        pbar.update()

### Parse the text

In [ ]:
def parseSpeechText(speeches):
    '''Run CLTK NLP pipeline to parse all the speeches'''
    
    pbar = NotebookPBar(max=len(speeches), prefix='Running NLP')

    for s in speeches:
        if not hasattr(s, 'passage') or s.passage is None:
            print('no passage:', s)
        elif not hasattr(s.passage, 'cltk') or s.passage.cltk is None:
            s.passage.runCltkPipeline(remove_punct=True)
        pbar.update()

### Format tokens as a table

In [ ]:
def makeTokenTable(speeches):
    '''Create a DataFrame with one row per token'''
    words = pd.DataFrame(dict(
        speech_id = s.id,
        book = s.l_fi.split('.')[0],
        line = s.passage.line_array[s.passage.getLineIndex(w)]['n'],
        l_ind = s.passage.getLineIndex(w)+1,
        spkr = s.getSpkrString(),
        addr = s.getAddrString(),
        gend_spkr = ','.join(sorted(set(inst.gender for inst in s.spkr))),
        gend_addr = ','.join(sorted(set(inst.gender for inst in s.addr))),
        string = w.string,
        lemma = w.lemma,
        upos = w.upos,
        is_voc = 'vocative' in str(w.features),
        features = str(w.features),
    ) for s in speeches for w in s.passage.cltk)

    # filter out punctuation tokens
    words = words[(words.string != '.') & (words.upos != 'PUNCT')]
        
    return words

## 3. Run the whole workflow

In [ ]:
speeches = {}
words = {}
works = ['iliad', 'odyssey', 'posthomerica']

In [ ]:
for work in works:
    print('Processing', work.title())
    
    # local file paths
    cache = os.path.join('data', f'{work}_speeches.pickle')
    output = os.path.join('data', f'{work}.csv')
    
    # use cached data if present
    if os.path.exists(cache):
        with open(cache, 'rb') as f:
            speeches[work] = pickle.load(f)
        print('loaded', len(speeches[work]), 'cached results')
    else:    
        speeches[work] = dlSpeechData(work)
        dlSpeechText(speeches[work])
        parseSpeechText(speeches[work])
        with open(cache, 'wb') as f:
            pickle.dump(speeches[work], f)
        print('saved', len(speeches[work]), 'results to', cache)
    
    # generate tabular data
    words[work] = makeTokenTable(speeches[work])
    
    # save output
    print(f'Writing {output}')
    words[work].to_csv(output, index=False)

### inspect the table of values

In [ ]:
words['iliad']

# 4. summary statistics

## helper functions

### Simple count of vocatives by book

In [ ]:
def getTableByBook(work):
    df = words[work][words[work].is_voc].pivot_table(
        index = 'book',
        values = 'speech_id',
        aggfunc = 'count',
        sort = False,
        fill_value = 0,
    )
    return df

def getPlotByBook(work):
    df = tableByBook(work)
    plot = df.plot.bar(
        title = f'Vocatives in the {work.title()}',
        legend = False,
        rot = False,
        ylabel = 'count',
        figsize = (10,4),
    )
    return plot

### Normalized for book length

In [ ]:
def getTableByBookNorm(work):

    df = words[work].pivot_table(
        index = 'book',
        values = 'speech_id',
        columns = 'is_voc',
        aggfunc = 'count',
        sort = False,
        fill_value = 0,
    ).rename(columns={True:'voc', False:'other'})

    df['prop'] = df['voc'] / (df['voc'] + df['other']) * 1000

    return df

def getPlotByBookNorm(work):
    df = getTableByBookNorm(work)
    plot = df['prop'].plot.bar(
        title = f'Vocatives in the {work.title()}',
        legend = False,
        ylabel = 'count per 1000 words',
        rot = False,
        figsize = (10,4),
    )
    
    return plot

### by speaker

In [ ]:
def getTableBySpeaker(work):
    
    df = words[work].pivot_table(
        index = 'spkr',
        values = 'speech_id',
        columns = 'is_voc',
        aggfunc = 'count',
        fill_value = 0,
    )

    df = df.rename(columns={True:'voc', False:'other'})

    df['prop'] = round(df['voc'] / (df['voc'] + df['other']) * 1000, 2)
    
    return df

### by part of speech

In [ ]:
def getTableByPOS(work):

    df = words[work].pivot_table(
        index = 'upos',
        values = 'speech_id',
        columns = 'is_voc',
        aggfunc = 'count',
        fill_value = 0,
    ).rename(columns={True:'voc', False:'other'})

    df.sort_values('voc', ascending=False)

## Display results

### Normalized vocatives by book

In [ ]:
display(interactive(getPlotByBookNorm, work=works))

### By Speaker

In [ ]:
view = lambda work: display(getTableBySpeaker(work))
interactive(view, work=works)

#### greatest number

In [ ]:
view = lambda work, n: display(getTableBySpeaker(work).sort_values('voc', ascending=False)[:n])
display(interactive(view, work=works, n=widgets.IntSlider(min=10, max=100, step=5, value=10)))

#### highest proportion

In [ ]:
view = lambda work, n: display(getTableBySpeaker(work).sort_values('prop', ascending=False)[:n])
display(interactive(view, work=works, n=widgets.IntSlider(min=10, max=100, step=5, value=10)))

#### highest proportion among speakers of at least 1000 words

In [ ]:
def view(work, min_words, n_results):
    df = getTableBySpeaker(work)
    df = df[(df.other + df.voc) >= min_words]
    df = df.sort_values('prop', ascending=False)[:n_results]
    display(df)

display(interactive(view, work=works, n_results=widgets.IntSlider(min=10, max=100, step=5, value=10),
                   min_words = widgets.IntSlider(min=100, max=2000, step=100, value=1000)))